In [1]:
import cv2
import mediapipe
import numpy as np

In [2]:
from time import sleep

In [3]:
import keyboard

### Detecting Hand

In [4]:
mp_hands=mediapipe.solutions.hands
mp_draw=mediapipe.solutions.drawing_utils

In [83]:
# cap=cv2.VideoCapture(0)

# hands=mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5)

# while cap.isOpened():
#     ret,frame=cap.read()
    
#     image=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
#     image=cv2.flip(image,1)
#     results=hands.process(image)
#     image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
#     if results.multi_hand_landmarks:
#         for num,hand in enumerate(results.multi_hand_landmarks):
#             mp_draw.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
#                                    mp_draw.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
#                                      mp_draw.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
        
#     cv2.imshow("hands",image)
    
#     if cv2.waitKey(1) & 0xFF==ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

### building keyboard

In [104]:
keys=[['1', '2', '3', '4', '5', '6' ,'7', '8', '9', '0'], 
      ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'],
      ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L','?'],
      ['Z', 'X', 'C', 'V', 'B', 'N', 'M', '.','space','backspace']]

In [6]:
buttons={}

#### building a keyboard on image

In [110]:
def build_keys(image,keys,itr=0,itr2=0,p1=[30,30],p2=[80,80]):
    if itr2==10:
        p1[0]=30
        p2[0]=80
        p1[1]+=60
        p2[1]+=60
        itr+=1
        itr2=0
        
    if itr<len(keys):
        
        if keys[itr][itr2]=='space':
            text='spc'
        elif keys[itr][itr2]=='backspace':
            text='bksp'
        else:
            text=keys[itr][itr2]
            
        cv2.rectangle(image,(p1[0],p1[1]),(p2[0],p2[1]),(150,100,50),cv2.FILLED)
        cv2.putText(image,text,(p1[0]+10,p2[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
        
        
        text=keys[itr][itr2]
        buttons[text]=[p1[0],p1[1]] ## x,y

        return build_keys(image,keys,itr,itr2+1,[p1[0]+60,p1[1]],[p2[0]+60,p2[1]])
    else:
        return image

#### getting the coordinates of index and middle finger tip

In [8]:
def get_coords(hands,results):
    coords=None
    for idx,classification in enumerate(results.multi_handedness):
        index=list(np.multiply(np.array((hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                                           hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y)),[640,480]).astype(int))

        middle=list(np.multiply(np.array((hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                                           hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y)),[640,480]).astype(int))
        coords=[index,middle]
    return coords

#### finding the nearest key of keyboard corresponding to finger position

In [82]:
def identify_point_key(point,buttons,keys):
    x=point[0]
    y=point[1]
    ans=None
    
    temp_x=0
    temp_y=0
    
    for key,val in buttons.items():
        if y > val[1] and val[1]>temp_y:  ### updating value of temp_y if y of point is greater than y coord of key
            temp_y=val[1]
            
        if x > val[0] and val[0] > temp_x: ### updating value of temp_x if x of point is greater than x coord of key
            temp_x=val[0]
    
            
    for key,val in buttons.items():
        if [temp_x,temp_y]==val:
            ans=key
    
    return ans,[temp_x,temp_y]

#### checking if key is selected

In [81]:
def isClicked(result,hand):
    index=0
    for idx,classification in enumerate(results.multi_handedness):
        if classification.classification[0].index==0:
            index=0
        else:
            index=1
        tip=np.multiply(hand.landmark[mp_hands.HandLandmark.THUMB_TIP].x,640) ### tip of thumb
        mcp=np.multiply(hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x,640) ### mcp of index finger
    
    if tip>=mcp and index==1: ### right hand
        return True
    if tip<=mcp and index==0: ### left hand
        return True
    else:
        return False

In [111]:
text=""
cap=cv2.VideoCapture(0)

hands=mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5)

while cap.isOpened():
    ret,frame=cap.read()
    
    image=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
    image=cv2.flip(image,1)
    results=hands.process(image)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    
    build_keys(image,keys)
    # cv2.rectangle(image,(30,270),(570,340),(150,100,50),cv2.FILLED)
    
    if results.multi_hand_landmarks:
        for num,hand in enumerate(results.multi_hand_landmarks):
            mp_draw.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                                   mp_draw.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                     mp_draw.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
            
            if get_coords(hand,results):
                coords=get_coords(hand,results)
                # print(coords[0])
                index=coords[0]
                middle=coords[1]
                x1,y1,x2,y2=index[0],index[1],middle[0],middle[1]
                point=[(x1+x2)//2,(y1+y2)//2] ### finding midpoint of index and middle finger tip
                cv2.circle(image,point,10,(0,0,255),cv2.FILLED)
                # print(point)
                
                ans,pos=identify_point_key(point,buttons,keys)
                if ans:
                    # print(pos,ans)
                    cv2.rectangle(image,(pos[0],pos[1]),(pos[0]+50,pos[1]+50),(0,0,255),cv2.FILLED)
                
                if isClicked(results,hand): 
                    
                    cv2.rectangle(image,(pos[0],pos[1]),(pos[0]+50,pos[1]+50),(0,255,0),cv2.FILLED)
                    # sleep(0.1)
                    # print(ans)
                    if ans and('A'<=ans<='Z' or '0'<=ans<='9'):
                        sleep(0.15)
                        keyboard.write(ans) ## for alphabets and numbers
                    else:
                        if ans:
                            sleep(0.15)
                            keyboard.press_and_release(ans) ## for keys like space, enter, backspace
        # cv2.putText(image,text,(50,310),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
        
    cv2.imshow("hands",image)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
        
cap.release()

cv2.destroyAllWindows()

In [77]:
# cap.release()
# cv2.destroyAllWindows()

In [12]:
buttons['Q'],buttons['1']

([90, 90], [30, 30])

In [13]:
buttons

{'1': [30, 30],
 '2': [90, 30],
 '3': [150, 30],
 '4': [210, 30],
 '5': [270, 30],
 '6': [330, 30],
 '7': [390, 30],
 '8': [450, 30],
 '9': [510, 30],
 '0': [30, 90],
 'Q': [90, 90],
 'W': [150, 90],
 'E': [210, 90],
 'R': [270, 90],
 'T': [330, 90],
 'Y': [390, 90],
 'U': [450, 90],
 'I': [510, 90],
 'O': [570, 90],
 'P': [30, 150],
 'A': [90, 150],
 'S': [150, 150],
 'D': [210, 150],
 'F': [270, 150],
 'G': [330, 150],
 'H': [390, 150],
 'J': [450, 150],
 'K': [510, 150],
 'L': [570, 150],
 '?': [30, 210],
 'Z': [90, 210],
 'X': [150, 210],
 'C': [210, 210],
 'V': [270, 210],
 'B': [330, 210],
 'N': [390, 210],
 'M': [450, 210],
 '.': [510, 210],
 'space': [570, 210],
 'backspace': [30, 270]}